<a href="https://colab.research.google.com/github/ykitaguchi77/ROP_AI_project/blob/master/Inference_for_cvat_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

# CUDAが使えるかどうかを確認
cuda_available = torch.cuda.is_available()

if cuda_available:
    print(f"CUDA is available! GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")


CUDA is not available.


#**Video2Img**

In [ ]:
import cv2
import os
from tqdm import tqdm

def extract_frames(video_path, output_dir):
    # 出力ディレクトリの作成
    os.makedirs(output_dir, exist_ok=True)

    # 動画の読み込み
    cap = cv2.VideoCapture(video_path)

    # 総フレーム数の取得
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # tqdmで進捗バーを表示
    with tqdm(total=total_frames, desc='フレーム抽出中') as pbar:
        frame_count = 0

        while True:
            # フレームの読み込み
            ret, frame = cap.read()

            # 読み込みが終了したらループを抜ける
            if not ret:
                break

            # フレームを保存
            output_path = os.path.join(output_dir, f'{os.path.basename(output_dir)}_{frame_count:04d}.jpg')
            cv2.imwrite(output_path, frame)

            frame_count += 1
            pbar.update(1)

    # キャプチャの解放
    cap.release()

    print(f'\n合計 {frame_count} フレームを抽出しました')

def main():
    # 入力動画のパス
    video_path = r'/content/drive/MyDrive/Deep_learning/ROP_AI_project/ROP_movie/IMG_1703.mp4'

    # 出力ディレクトリ
    output_dir = r'/content/drive/MyDrive/Deep_learning/ROP_AI_project/ROP_images/IMG_1703'

    # フレーム抽出の実行
    extract_frames(video_path, output_dir)

if __name__ == '__main__':
    main()

フレーム抽出中: 100%|██████████| 888/888 [00:45<00:00, 19.72it/s]


合計 888 フレームを抽出しました


# **1枚画像**

In [ ]:
# prompt: gdrive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install ultralytics
!pip install wget

In [ ]:

from ultralytics import RTDETR
import wget
import cv2
import matplotlib.pyplot as plt

# モデルが未定義ならロードする
try:
    model  # 変数modelが定義されているか確認
except NameError:
    print("modelが未定義のため、ロードを実行します。")
    model = RTDETR(r"/content/drive/MyDrive/Deep_learning/ROP_AI_project/models/rtdetr-l-1700_1703.pt")

# 推論を実行: YOLOの推論
image_path = r'/content/drive/MyDrive/Deep_learning/ROP_AI_project/ROP_images/IMG_1703/IMG_1703_0024.jpg'
results = model(image_path, save=False)

for r in results:
    boxes = r.boxes  # YOLOv8の場合

    # YOLO形式出力
    for box in boxes:
        cls_id = int(box.cls[0])  # クラスID
        x_center, y_center, width, height = box.xywhn[0]

        # YOLO形式: class x_center y_center width height
        # 小数点以下6桁で出力したい場合
        print(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    #検出結果を画像に描画して表示
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(r.plot(), cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # 結果の保存
    #cv2.imwrite('result.jpg', r.plot())

In [ ]:
from ultralytics import RTDETR
import cv2
import os

# 出力ディレクトリの作成
output_dir = '/content/video/output'
os.makedirs(output_dir, exist_ok=True)

# モデルは既にロード済みと仮定
try:
    model
except NameError:
    model = RTDETR(r"/content/drive/MyDrive/Deep_learning/ROP_AI_project/models/rtdetr-l-1700_1703.pt")

# 入力動画のパス
video_path = r'/content/drive/MyDrive/Deep_learning/ROP_AI_project/ROP_movie/IMG_1623_16cm(福嶋）.mp4'
output_path = os.path.join(output_dir, 'result.mp4')

# 入力動画を開く
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Error: 動画ファイル '{video_path}' を開けませんでした")
    exit()

# 動画の基本情報を取得
target_width = 224
original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# アスペクト比を保持してリサイズ
aspect_ratio = original_width / original_height
target_height = int(target_width / aspect_ratio)

# 出力用のビデオライター設定
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (target_width, target_height))

try:
    frame_count = 0
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # フレーム処理の進捗を表示
        frame_count += 1
        if frame_count % 10 == 0:  # 10フレームごとに進捗を表示
            print(f"Processing: {frame_count}/{total_frames} frames ({(frame_count/total_frames*100):.1f}%)")

        # リサイズ
        frame = cv2.resize(frame, (target_width, target_height))

        # 推論実行
        results = model(frame)

        # 結果を描画
        annotated_frame = results[0].plot()

        # フレームを書き出し
        out.write(annotated_frame)

except KeyboardInterrupt:
    print("\n処理が中断されました")
finally:
    # リソースの解放
    cap.release()
    out.release()

print(f"\n処理が完了しました。\n保存先: {output_path}")

# **複数画像**

階層構造
```
archive.zip/
   ├── data.yaml  # configuration file
   ├── train.txt  # list of train subset image paths
   │
   ├── images/
   │   ├── train/  # directory with images for train subset
   │   │    ├── image1.jpg
   │   │    ├── image2.jpg
   │   │    ├── image3.jpg
   │   │    └── ...
   ├── labels/
   │   ├── train/  # directory with annotations for train subset
   │   │    ├── image1.txt
   │   │    ├── image2.txt
   │   │    ├── image3.txt
   │   │    └── ...

```

***.txt (ラベル)
```
0 0.260745 0.535950 0.218015 0.030600
1 0.760795 0.533850 0.213035 0.043800
```

daya.yaml
```
names:
  0: Right_eye
  1: Left_eye
path: .
train: train.txt
```

train.txt
```
data/images/train/***.jpg
data/images/train/***.jpg
...(pathの羅列。拡張子を合わせる)
```
※ラベルや画像と対応を合わせること



In [ ]:
import os
import glob
import shutil
from tqdm import tqdm
from ultralytics import RTDETR

def create_yolo_dataset(orig_image_dir, output_dir, model_path):
    """
    YOLOデータセットの構造を作成する

    Args:
        orig_image_dir: 元の画像が存在するディレクトリ
        output_dir: 出力先のディレクトリ
        model_path: RTDETRモデルのパス
    """
    # 進捗バーの設定
    progress = tqdm(
        total=4,
        desc="データセット作成",
        bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'
    )

    # 必要なディレクトリを作成
    os.makedirs(os.path.join(output_dir, "labels", "train"), exist_ok=True)
    progress.update(1)

    # data.yamlの作成
    yaml_content = """names:
  0: Right_eye
  1: Left_eye
path: .
train: train.txt"""

    with open(os.path.join(output_dir, "data.yaml"), 'w', encoding='utf-8') as f:
        f.write(yaml_content)
    progress.update(1)

    # モデルのロード
    try:
        model = RTDETR(model_path)
    except Exception as e:
        print(f"モデルのロードに失敗: {e}")
        return

    # 画像ファイルのリストを取得
    image_files = []
    extensions = ['*.jpg', '*.JPG', '*.jpeg', '*.JPEG', '*.png', '*.PNG', '*.tif', '*.TIF', '*.tiff', '*.TIFF']

    for ext in extensions:
        image_files.extend(glob.glob(os.path.join(orig_image_dir, ext)))

    total_files = len(image_files)
    print(f"\n合計 {total_files} 個のファイルが見つかりました")

    # 推論とラベル生成の進捗バー
    inference_progress = tqdm(
        total=total_files,
        desc="推論とラベル生成",
        bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'
    )

    # train.txtの作成とラベルファイルの生成
    with open(os.path.join(output_dir, "train.txt"), 'w', encoding='utf-8') as f:
        for image_path in image_files:
            # 元の拡張子を保持
            basename = os.path.basename(image_path)
            name_without_ext = os.path.splitext(basename)[0]

            # 推論を実行
            results = model(image_path, save=False)

            # ラベルファイルのパス（train/配下に.txt）
            label_path = os.path.join(output_dir, "labels", "train", f"{name_without_ext}.txt")

            # YOLO形式の推論結果をテキストファイルに書き込み
            with open(label_path, 'w', encoding='utf-8') as lf:
                for r in results:
                    boxes = r.boxes
                    for box in boxes:
                        cls_id = int(box.cls[0])
                        x_center, y_center, width, height = box.xywhn[0]
                        line = f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"
                        lf.write(line)

            # train.txtにYOLOフォーマットのパスを書き込み
            yolo_path = f"data/images/train/{basename}"
            f.write(f"{yolo_path}\n")
            inference_progress.update(1)

    inference_progress.close()
    progress.update(1)

    # ZIP圧縮
    print("\nZIP圧縮を開始します...")
    zip_path = output_dir + ".zip"
    total = sum(len(files) for _, _, files in os.walk(output_dir))

    with tqdm(total=total, desc="ZIP圧縮", bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]') as pbar:
        for root, _, files in os.walk(output_dir):
            for file in files:
                pbar.update(1)
        shutil.make_archive(os.path.splitext(zip_path)[0], 'zip', output_dir)

    progress.update(1)
    progress.close()
    print(f"\nフォルダを {zip_path} に圧縮しました。")

# 使用例
if __name__ == "__main__":
    orig_image_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\image"
    output_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\archive"
    model_path = r"C:\Users\ykita\FacePhoto_instance\models\135best_rtdetr.pt"

    create_yolo_dataset(orig_image_dir, output_dir, model_path)

In [ ]:
import os
import glob
from ultralytics import RTDETR
from tqdm import tqdm  # tqdmライブラリをインポート

# 1) 画像のフォルダを指定する
orig_image_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\image"
# 2) 出力先のフォルダを指定する。フォルダがなければ作成する
dst_parent_dir= r"C:\Users\ykita\FacePhoto_instance\inference\201-295\archive" #####
dst_label_dir = os.path.join(dst_parent_dir, "labels", "train")
os.makedirs(dst_label_dir, exist_ok=True)

# モデルが未定義ならロードする
try:
    model  # 変数modelが定義されているか確認
except NameError:
    print("modelが未定義のため、ロードを実行します。")
    model_path = r"C:\Users\ykita\FacePhoto_instance\models\135best_rtdetr.pt"
    model = RTDETR(model_path)

# 処理対象とする画像の拡張子
image_extensions = [
    '.jpg', '.jpeg', '.JPG', '.JPEG',
    '.png', '.PNG',
    '.tif', '.tiff', '.TIF', '.TIFF'
]

# 指定フォルダ配下の該当画像ファイルをすべて取得
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(orig_image_dir, f'*{ext}')))

# tqdmで進捗状況を表示しながらループ
for image_path in tqdm(image_files, desc='推論中', unit='枚'):
    # 推論を実行
    results = model(image_path, save=False)

    # 画像と同じファイル名（拡張子を除く）でテキストファイル出力
    basename = os.path.splitext(os.path.basename(image_path))[0]
    save_txt_path = os.path.join(dst_label_dir, f"{basename}.txt")

    # YOLO形式の推論結果をテキストファイルに書き込み
    with open(save_txt_path, 'w', encoding='utf-8') as f:
        for r in results:
            boxes = r.boxes  # YOLOv8系の出力
            for box in boxes:
                cls_id = int(box.cls[0])
                x_center, y_center, width, height = box.xywhn[0]
                # YOLO形式: class x_center y_center width height（小数点以下6桁）
                line = f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"
                f.write(line)

print("すべての画像に対する推論が完了し、YOLO形式でラベルを出力しました。")


## **CVAT仕様のフォルダ構成に**

In [ ]:
import os
import glob

# 現在のlabelsフォルダのパス
orig_image_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\image"
dst_parent_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\archive"


# yamlとimg_listのパスを定義
data_yaml_path = os.path.join(dst_parent_dir, "data.yaml")
img_list_path = os.path.join(dst_parent_dir, "train.txt")

# data.yaml を書き込み
with open(data_yaml_path, 'w', encoding='utf-8') as f:
    f.write("names:\n")
    f.write("  0: Right_eye\n")
    f.write("  1: Left_eye\n")
    f.write("path: .\n")
    f.write("train: train.txt\n")

# labels_dir配下の .txt ファイルから、img_list.txt を作成
txt_files = glob.glob(os.path.join(dst_label_dir, "*.txt"))

with open(img_list_path, 'w', encoding='utf-8') as f:
    for txt_file in txt_files:
        basename = os.path.splitext(os.path.basename(txt_file))[0]
        f.write(f"data/images/train/{basename}.jpg\n")

print("data.yaml と img_list.txt が、labelsフォルダと同じ階層に作成されました。")

data.yaml と img_list.txt が、labelsフォルダと同じ階層に作成されました。


In [ ]:
import shutil

#dst_parent_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\archive"
# dst_parent_dirをzip圧縮
zip_path = dst_parent_dir + ".zip"
shutil.make_archive(dst_parent_dir, 'zip', dst_parent_dir)

print(f"フォルダを {zip_path} に圧縮しました。")


フォルダを C:\Users\ykita\FacePhoto_instance\inference\201-295\archive.zip に圧縮しました。
